<a href="https://colab.research.google.com/github/fecsaba/automatic_photo/blob/main/auto_get_meta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import PngImagePlugin, Image
import os

class PngMetadataReader:
    """PNG fájlok metaadatainak kezelésére szolgáló osztály"""
    
    def __init__(self, directory_path=None):
        """
        Inicializálja az osztályt
        
        Args:
            directory_path (str, optional): A mappa elérési útja
        """
        self.directory_path = directory_path
        self.image_path = None
    
    def set_directory_path(self, directory_path):
        """
        Beállítja a feldolgozandó mappa útvonalát
        
        Args:
            directory_path (str): A mappa elérési útja
        """
        self.directory_path = directory_path
    
    def read_metadata(self):
        """
        Beolvassa és megjeleníti egy PNG fájl metaadatait
        
        Returns:
            dict: A metaadatok szótár formátumban, vagy None hiba esetén
        """
        if not self.image_path:
            print("Nincs megadva képfájl útvonal!")
            return None
            
        try:
            # Megnyitjuk a képet
            with Image.open(self.image_path) as img:
                # Ellenőrizzük, hogy PNG fájl-e
                if isinstance(img, PngImagePlugin.PngImageFile):
                    metadata = img.text
                    if metadata:
                        print(f"\nMetaadatok a következő fájlhoz: {os.path.basename(self.image_path)}")
                        print("-" * 50)
                        for k, v in metadata.items():
                            print(f"**{k}**:\n{v}\n")
                        return metadata
                    else:
                        print(f"\nNem találtunk metaadatokat a következő fájlban: {os.path.basename(self.image_path)}")
                        return {}
                else:
                    print(f"\nA fájl nem PNG, vagy nem PngImageFile típusú: {os.path.basename(self.image_path)}")
                    return None
        except FileNotFoundError:
            print(f"A fájl nem található: {self.image_path}")
            return None
        except Exception as e:
            print(f"Hiba történt a következő fájlnál {os.path.basename(self.image_path)}: {str(e)}")
            return None
    
    def process_directory(self):
        """
        Feldolgozza a megadott mappában található összes PNG fájlt
        
        Returns:
            dict: A fájlnevek és a hozzájuk tartozó metaadatok szótára
        """
        if not self.directory_path:
            print("Nincs megadva mappa útvonal!")
            return None
            
        if not os.path.exists(self.directory_path):
            print(f"A megadott mappa nem létezik: {self.directory_path}")
            return None
            
        results = {}
        print(f"\nPNG fájlok keresése a következő mappában: {self.directory_path}")
        print("=" * 70)
        
        # Végigmegyünk a mappa tartalmán
        for filename in os.listdir(self.directory_path):
            if filename.lower().endswith('.png'):
                self.image_path = os.path.join(self.directory_path, filename)
                metadata = self.read_metadata()
                if metadata:
                    results[filename] = metadata
        
        if not results:
            print("\nNem találtunk PNG fájlokat a megadott mappában.")
        
        return results

# Példa használat
if __name__ == "__main__":
    # Példa a használatra
    reader = PngMetadataReader("/content/drive/MyDrive/Image/")
    all_metadata = reader.process_directory()
    
    # Összesítés
    if all_metadata:
        print("\nÖsszesítés:")
        print("=" * 70)
        print(f"Összesen {len(all_metadata)} PNG fájl metaadatait olvastuk be.")